In [1]:
from selenium import webdriver

### 1. 크롬 드라이버 열기

In [2]:
url = "https://stats.nba.com/events/?flag=3&CFID=155&CFPARAMS=2018-19&TeamID=1610612744&ContextMeasure=FGA&Season=2018-19&section=team&sct=hex"

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get(url)

### 2. 컬럼 데이터 크롤링

In [5]:
# 컬럼 엘리먼트 선택
column_elements = driver.find_elements_by_css_selector(".nba-stat-table__overflow > table > thead > tr > th")
len(column_elements)

13

In [6]:
# 컬럼 엘리먼트에서 데이터 수집
columns = [column.text for column in column_elements[1:]]
columns

['PLAYER',
 'PLAY TYPE',
 'MADE',
 'SHOT TYPE',
 'BOXSCORE',
 'VTM',
 'HTM',
 'GAME DATE',
 'PERIOD',
 'TIME REMAINING',
 'SHOT DISTANCE (FT)',
 'TEAM']

### 3. 선수 데이터 크롤링

In [7]:
# 선수별 데이터 엘리먼트 선택
data_elements = driver.find_elements_by_css_selector(".nba-stat-table__overflow > table > tbody > tr")
len(data_elements)

50

In [8]:
%%time
# 선수별 엘리먼트에서 데이터 수집
datas = []
for element in data_elements:
    data = {}
    for idx, column in enumerate(columns):        
        data[column] = element.find_element_by_css_selector("td:nth-child({})".format(idx+2)).text
    datas.append(data)

In [9]:
# 데이터 프레임으로 만듦
df = pd.DataFrame(datas)
df = df[columns]
df.tail()

,PLAYER,PLAY TYPE,MADE,SHOT TYPE,BOXSCORE,VTM,HTM,GAME DATE,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),TEAM
45,Draymond Green,Fadeaway Jump Shot,Missed Shot,2PT Field Goal,Boxscore,OKC,GSW,2018-10-16,3,10:07,10,Golden State Warriors
46,Stephen Curry,Running Pull-Up Jump Shot,Made Shot,3PT Field Goal,Boxscore,OKC,GSW,2018-10-16,3,09:47,28,Golden State Warriors
47,Kevin Durant,Jump Shot,Missed Shot,3PT Field Goal,Boxscore,OKC,GSW,2018-10-16,3,09:11,22,Golden State Warriors
48,Damian Jones,Tip Dunk Shot,Made Shot,2PT Field Goal,Boxscore,OKC,GSW,2018-10-16,3,09:08,0,Golden State Warriors
49,Kevin Durant,Pullup Jump shot,Made Shot,2PT Field Goal,Boxscore,OKC,GSW,2018-10-16,3,08:29,18,Golden State Warriors


### 4. 페이지 이동

In [17]:
# 전체 페이지의 갯수
page_elements = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option")
total_pages = len(page_elements) - 1
total_pages

148

In [18]:
# 페이지 이동
page = 5
page_elements[page].click()

### 5. 브라우져 종료

In [11]:
# 브라우져 종료
# driver.quit()

### 6. 함수로 만들어서 모든 데이터 크롤링

In [19]:
# 드라이버 만들기
def make_driver():
    url = "https://stats.nba.com/events/?flag=3&CFID=155&CFPARAMS=2018-19&TeamID=1610612744&ContextMeasure=FGA&Season=2018-19&section=team&sct=hex"
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

In [20]:
# 컬럼 데이터 크롤링
def crawling_columns(driver):
    column_elements = driver.find_elements_by_css_selector(".nba-stat-table__overflow > table > thead > tr > th")
    return [column.text for column in column_elements[1:]]

In [29]:
# 선수 데이터 크롤링 후 데이터 프레임으로 리턴
def crawling_players(driver, columns):
    data_elements = driver.find_elements_by_css_selector(".nba-stat-table__overflow > table > tbody > tr")
    datas = []
    for element in data_elements:
        data = {}
        for idx, column in enumerate(columns):        
            data[column] = element.find_element_by_css_selector("td:nth-child({})".format(idx+2)).text
        datas.append(data)
        
    df = pd.DataFrame(datas)
    return df[columns]

In [23]:
# 전체 페이지 확인
def total_page(driver):
    page_elements = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option")
    return len(page_elements) - 1

In [22]:
# 페이지 이동
def move_page(driver, page):
    page_elements = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option")
    page_elements[page].click()

In [24]:
# 전체 페이지확인
driver = make_driver()
result = total_page(driver)
driver.quit()
print(result)

148


In [25]:
driver = make_driver()

In [26]:
columns = crawling_columns(driver)

In [27]:
player_dfs = []

In [30]:
start, end = 1, 2
for page in range(start, end + 1):
    player_df = crawling_players(driver, columns)
    player_dfs.append(player_df)
    move_page(driver, page)

In [31]:
df = pd.concat(player_dfs, ignore_index=True, sort=False)
df.tail()

,PLAY TYPE,MADE,SHOT TYPE,BOXSCORE,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),TEAM
95,Fadeaway Jump Shot,Missed Shot,2PT Field Goal,Boxscore,3,10:07,10,Golden State Warriors
96,Running Pull-Up Jump Shot,Made Shot,3PT Field Goal,Boxscore,3,09:47,28,Golden State Warriors
97,Jump Shot,Missed Shot,3PT Field Goal,Boxscore,3,09:11,22,Golden State Warriors
98,Tip Dunk Shot,Made Shot,2PT Field Goal,Boxscore,3,09:08,0,Golden State Warriors
99,Pullup Jump shot,Made Shot,2PT Field Goal,Boxscore,3,08:29,18,Golden State Warriors


In [33]:
driver.quit()

In [32]:
# 페이지 설정하면 설정한 만큼 데이터 크롤링
def run(start, end):
    driver = make_driver()
    columns = crawling_columns(driver)
    player_dfs = []
    for page in range(start, end + 1):
        player_df = crawling_players(driver, columns)
        player_dfs.append(player_df)
        move_page(driver, page)
    driver.quit()
    return pd.concat(player_dfs, ignore_index=True, sort=False)

In [34]:
%%time
result_df = run(1, 10)

CPU times: user 6.21 s, sys: 360 ms, total: 6.57 s
Wall time: 2min 47s


In [35]:
result_df.tail()

,PLAYER,PLAY TYPE,MADE,SHOT TYPE,BOXSCORE,VTM,HTM,GAME DATE,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),TEAM
495,Stephen Curry,Cutting Layup Shot,Made Shot,2PT Field Goal,Boxscore,GSW,NYK,2018-10-26,1,09:01,0,Golden State Warriors
496,Kevin Durant,Running Dunk Shot,Made Shot,2PT Field Goal,Boxscore,GSW,NYK,2018-10-26,1,08:35,0,Golden State Warriors
497,Klay Thompson,Driving Floating Jump Shot,Missed Shot,2PT Field Goal,Boxscore,GSW,NYK,2018-10-26,1,07:35,5,Golden State Warriors
498,Klay Thompson,Jump Shot,Missed Shot,3PT Field Goal,Boxscore,GSW,NYK,2018-10-26,1,06:59,22,Golden State Warriors
499,Klay Thompson,Step Back Jump shot,Made Shot,2PT Field Goal,Boxscore,GSW,NYK,2018-10-26,1,06:38,18,Golden State Warriors
